In [1]:
# Import dependencies
print('Importing libraries')
import pandas as pd
import zipfile
import json
import requests as req
from datetime import datetime
print('Done importing')

Importing libraries
Done importing


In [2]:
# Load datafram from csv compressed in the zip file

print('Loading source file: %s' % datetime.now())

data_zip_path = 'data/Police_Incidents.zip'
zf = zipfile.ZipFile(data_zip_path) # having Police_Incidents.csv zipped file.
police_incidents_df = pd.read_csv(zf.open('Police_Incidents.csv'))
print('File loaded: %s' % datetime.now())


Loading source file: 2018-01-16 20:11:21.918631
File loaded: 2018-01-16 20:11:36.768243


In [6]:
# Function  get_ymd helps us to get year, month, and day   

def get_ymd(data):
    
    months = []
    years = []
    days = []
    
    # For each row in the column Date
    for r_date in data['Date']:
         
        full_date = datetime.strptime(r_date, "%m/%d/%Y")
        months.append(str(full_date.month))
        years.append(str(full_date.year))
        days.append(str(full_date.day))
        
    # Create columns from the list
    data['Year'] = years
    data['Month'] = months
    data['Day_m'] = days


In [7]:
# Call function get_ymd for splitting column DATE in Year, Month and Day_m columns.

print('Converting dates: %s' % datetime.now())
get_ymd(police_incidents_df)   
print('Dates converted: %s' % datetime.now())

Converting dates: 2018-01-16 20:41:59.208905
Dates converted: 2018-01-16 20:42:40.207790


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,Year,Month,Day_m
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)",15006027571000,2015,1,19
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821003074,2015,2,1
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821004014,2015,2,1
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821015200,2015,2,1
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)",15009822628160,2015,1,27


In [8]:
# Drop columns Descript, Date, Address and PdId which we will not used for any purpose.

print('Dropping columns: %s' % datetime.now())
police_incidents_df.drop(['Descript', 'Date', 'Address', 'PdId'], axis=1, inplace= True)
print('Columns dropped: %s' % datetime.now())


Dropping columns: 2018-01-16 20:49:40.790245
Columns dropped: 2018-01-16 20:49:42.264590


(2165249, 12)

In [9]:
# Grab DataFrame rows where column years has certain values

print('Truncating by year: %s' % datetime.now())

years = ['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']
police_incidents_df = police_incidents_df[police_incidents_df.Year.isin(years)]

police_incidents_df_sorted = police_incidents_df.sort_values(by=['Year', 'Month', 'Day_m', 'Time'])

print('Data truncated: %s' % datetime.now())



Truncating by year: 2018-01-16 20:49:51.847386
Data truncated: 2018-01-16 20:49:54.508423


,IncidntNum,Category,DayOfWeek,Time,PdDistrict,Resolution,X,Y,Location,Year,Month,Day_m
247047,120511816,OTHER OFFENSES,Friday,00:01,NORTHERN,NONE,-122.420078,37.783521,"(37.7835214867196, -122.420078493883)",2010,1,1
247906,110651381,OTHER OFFENSES,Friday,00:01,CENTRAL,NONE,-122.409888,37.806026,"(37.8060255339956, -122.409888227377)",2010,1,1
263368,141053259,DISORDERLY CONDUCT,Friday,00:01,CENTRAL,NONE,-122.414011,37.786677,"(37.7866770011071, -122.414010814727)",2010,1,1
296994,106002257,LARCENY/THEFT,Friday,00:01,NORTHERN,NONE,-122.433799,37.805042,"(37.8050423291627, -122.433799454581)",2010,1,1
311263,101200525,BURGLARY,Friday,00:01,PARK,NONE,-122.447617,37.782584,"(37.7825841925678, -122.447617194265)",2010,1,1


In [10]:
# Rename the columns for future convenience

police_incidents_df_sorted.rename(columns={
    'IncidntNum': 'id',
    'Category': 'category',
    'DayOfWeek': 'day_w',
    'Time': 'time',
    'Resolution': 'resolution',
    'X': 'lng',
    'Y': 'lat',
    'Year': 'year',
    'Month': 'month',
    'Day_m': 'day_m',
    'PdDistrict': 'district',
    'Location': 'location'
}, inplace=True)

In [11]:
# Function map_crime helps get meta categories

def map_crime(cat):
    
    # Use this dictionary to add meta categories
    incident_map = {'BAD CHECKS': 'WC',
             'BRIBERY': 'WC',
             'EMBEZZLEMENT': 'WC',
             'EXTORTION' : 'WC',
             'FORGERY/COUNTERFEITING' : 'WC',
             'FRAUD' : 'WC',
             'SUSPICIOUS OCC' : 'WC',
             'ARSON' : 'BC',
             'ASSAULT' : 'BC',
             'BURGLARY' : 'BC',
             'DISORDERLY CONDUCT' : 'BC',
             'DRIVING UNDER THE INFLUENCE' : 'BC',
             'GAMBLING' : 'BC',
             'KIDNAPPING' : 'BC',
             'LARCENY/THEFT' : 'BC',
             'LIQUOR LAWS' : 'BC',
             'RECOVERED VEHICLE' : 'BC',
             'ROBBERY' : 'BC',
             'SEX OFFENSES, FORCIBLE' : 'BC',
             'STOLEN PROPERTY' : 'BC',
             'TRESPASS' : 'BC',
             'VANDALISM' : 'BC',
             'VEHICLE THEFT' : 'BC',
             'DRUG/NARCOTIC' : 'OI',
             'DRUNKENNESS' : 'OI',
             'FAMILY OFFENSES' : 'OI',
             'LOITERING' : 'OI',
             'MISSING PERSON' : 'OI',
             'NON-CRIMINAL' : 'OI',
             'OTHER OFFENSES' : 'OI',
             'PORNOGRAPHY/OBSCENE MAT' : 'OI',
             'PROSTITUTION' : 'OI',
             'RUNAWAY' : 'OI',
             'SECONDARY CODES' : 'OI',
             'SEX OFFENSES, NON FORCIBLE' : 'OI',
             'SUICIDE' : 'OI',
             'TREA' : 'OI',
             'WARRANTS' : 'OI',
             'WEAPON LAWS' : 'OI'}

    return incident_map[cat]

In [12]:
# Add meta categories

incident_list = []

print('Adding meta categories: %s' % datetime.now())
for i in police_incidents_df_sorted['category']:
    incident_list.append(map_crime(i))

police_incidents_df_sorted['meta_cat'] = incident_list
print('Meta categories added: %s' % datetime.now())

Adding meta categories: 2018-01-16 21:02:59.455191
Meta categories added: 2018-01-16 21:03:02.402301


In [13]:
# Rearrange columns 

police_incts_df = police_incidents_df_sorted[['id', 'year', 'month', 'day_m', 'day_w', 'time', 
                                              'category', 'meta_cat', 'resolution', 'location', 'lat', 'lng']]


In [16]:
# Trim down the dataframe to only what we need

print('Preparing coordinate-zipcode table: %s' % datetime.now())

incdt_loc_df= police_incts_df.drop_duplicates(subset= ['location'], keep="first")
coords_df = incdt_loc_df.filter(['location', 'lat', 'lng'], axis=1)
coords_df.reset_index(drop=True, inplace=True)
print('Setup ready: %s' % datetime.now())



Preparing coordinate-zipcode table: 2018-01-16 21:08:07.443109
Setup ready: 2018-01-16 21:08:07.514789


(52855, 3)

In [17]:
# Use this function to retrieve content of a file. Mainly used for getting api keys from a local file.
# It's assumed our file contains a single line, with our API key

def get_file_contents(filename):

    try:
        with open(filename, 'r') as f:
            
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


In [18]:
# Use this function to get the zipcode from a lat/lng pair

def get_zip_code(lat, lng):
    base_url = 'http://ws.geonames.net/findNearbyPostalCodesJSON?'
    full_url = base_url + 'lat=%s&lng=%s&username=%s' % (lat, lng, geonames_username)
    
    zipcode = req.get(full_url).json()['postalCodes'][0]['postalCode']
    
    return str(zipcode)

In [ ]:

geonames_username = get_file_contents('.geonames_username')

# Retrieve zip codes using API calls function
zip_list = []

print('Retrieving zipcodes')
for i in range(0, len(coords_df)):
    print('%s: %s' % (i, datetime.now()))
    zipcode = get_zip_code(coords_df['lat'][i], coords_df['lng'][i])
    zip_list.append(zipcode)
    
# Add zip codes to dataframe
coords_df['zipcode'] = zip_list


In [ ]:
# Export coordinates zipcode and compress it using zipfile library

print('Exporting: %s' % datetime.now())
coords_df.to_csv('data/coord-zipcode-table.csv', index=False)

coords_zip = zipfile.ZipFile('data/coord-zipcode-table.zip', 'w')
coords_zip.write('data/coord-zipcode-table.csv', compress_type=zipfile.ZIP_DEFLATED)
 
coords_zip.close()
print('All done: %s' % datetime.now())


In [33]:
#  Rename column 'name'of schools dataframe and merge both dataframes (tables) 

print('Add zipcode column: %s' % datetime.now())
data_merged_df = pd.merge(police_incts_df, coord_df, on="location")

# Delete duplicate columns and rename them
data_merged_df.drop(['lng_x', 'lat_y'], axis=1, inplace= True)
data_merged_df.rename(columns={'lat_x': 'lat', 'lng_y' : 'lng'}, inplace=True)
final_data_df = data_merged_df.sort_values(by=['year', 'month', 'day_m', 'time'])
final_data_df.reset_index(drop=True, inplace=True)

print('Done: %s' % datetime.now())

Add zipcode column: 2018-01-16 21:48:56.643940
Done: 2018-01-16 21:48:57.997996


In [42]:
# Export final main data file

print('Exporting: %s' % datetime.now()) 

final_data_df.to_csv('main-data.csv', index=False)

main_data_zip = zipfile.ZipFile('data/main-data.zip', 'w') 
main_data_zip.write('main-data.csv',compress_type=zipfile.ZIP_DEFLATED)

main_data_zip.close()

print('All done: %s' % datetime.now())

# final_data_df.to_csv('data/main-data.csv')


Exporting: 2018-01-16 22:07:59.948146
All done: 2018-01-16 22:08:23.522328
